In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import pickle
import os
import time
from sklearn.metrics import recall_score,precision_score,accuracy_score
%precision 3

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'%.3f'

In [2]:
def logistic_regression(X, y,n_classes,initializer=None,seed=42,learning_rate=0.01):
    n_inputs_including_bias = int(X.shape[1])
    with tf.name_scope("logistic_regression"):
        with tf.name_scope("model"):
            if initializer is None:
                initializer = tf.random_uniform([n_inputs_including_bias, n_classes], -1.0, 1.0, seed=seed)
            theta = tf.Variable(initializer, name="theta")
            logits = tf.matmul(X, theta, name="logits")
        with tf.name_scope("train"):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( 
                        labels=y, logits=logits)+beta*tf.nn.l2_loss(theta))             
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(loss)
            loss_summary = tf.summary.scalar('Cross-Entropy_Loss', loss)
        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
        with tf.name_scope("save"):
            saver = tf.train.Saver()
    return loss, training_op, loss_summary, init, saver,logits

In [3]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [4]:
X_train=np.load("obj/X_train.npy")
y_train=np.load("obj/y_train.npy")
X_test=np.load("obj/X_test.npy")
y_test=np.load("obj/y_test.npy")
X_devel=np.load("obj/X_devel.npy")
y_devel=np.load("obj/y_devel.npy")

In [5]:
def random_batch(X_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return X_batch, y_batch

In [6]:
def add_biastomatrix(*args):
    matrices=[]
    for X in args:
        m=X.shape[0]
        X=np.c_[np.ones((m,1)),X]
        matrices.append(X)
    return tuple(matrices)


In [7]:
(X_train_withbias,X_test_withbias,X_devel_withbias)=add_biastomatrix(X_train,X_test,X_devel)

In [8]:
print("Train data with bias shape:{}".format(X_train_withbias.shape))
print("Test data with bias shape:{}".format(X_test_withbias.shape))
print("Development data with bias shape:{}".format(X_devel_withbias.shape))

Train data with bias shape:(14823, 101)
Test data with bias shape:(1998, 101)
Development data with bias shape:(1998, 101)


In [9]:
beta=0.01
n_inputs = 100
logdir = log_dir("logreg")
n_classes=y_train.shape[1]
X = tf.placeholder(tf.float32, shape=(None, n_inputs + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, n_classes), name="y")
loss, training_op, loss_summary, init, saver,logits = logistic_regression(X, y,n_classes)
train_writer = tf.summary.FileWriter(logdir+"/train", tf.get_default_graph())
test_writer=tf.summary.FileWriter(logdir+"/test")
devel_writer=tf.summary.FileWriter(logdir+"/devel")
merged=tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
def calculateScores(logits,y):
    max_scores_indices=np.argmax(logits,axis=1)
    n_examples=logits.shape[0]
    predictions=np.zeros_like(logits,dtype=np.int)
    predictions[range(n_examples),max_scores_indices]=1
    del logits
    accuracy=accuracy_score(y,predictions)
    precision=precision_score(y,predictions,average="macro")
    recall=recall_score(y,predictions,average='macro')
    return accuracy,precision,recall

In [11]:
m=X_train.shape[0]
n_epochs = 1001
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
checkpoint_path = "/tmp/logistic_regressionl2.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_logreg_model"

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
    train_start_time=time.time()
    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(X_train_withbias, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        test_loss,summary_test = sess.run([loss,merged], feed_dict={X: X_test_withbias, y: y_test})
        test_writer.add_summary(summary_test, epoch)
        devel_loss,summary_devel=sess.run([loss,merged],feed_dict={X:X_devel_withbias,y:y_test})
        devel_writer.add_summary(summary_devel,epoch)
        if epoch % 100 == 0:
            train_loss,summary_train=sess.run([loss,merged],feed_dict={X:X_train_withbias,y:y_train})
            train_writer.add_summary(summary_train,epoch)
            print("Epoch:", epoch, "\tTest Loss:", test_loss,"\t Devel Loss:",
                  devel_loss,"\tTrain Loss:",train_loss)
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
    train_end_time=time.time()
    print("Time taken for training: ",train_end_time-train_start_time)
    saver.save(sess, final_model_path)
    os.remove(checkpoint_epoch_path)
    test_start_time=time.time()
    test_logits=sess.run(logits,feed_dict={X:X_test_withbias})
    devel_logits=sess.run(logits,feed_dict={X:X_devel_withbias})
    (test_accuracy,test_precision,test_recall)=calculateScores(np.array(test_logits),y_test)
    (devel_accuracy,devel_precision,devel_recall)=calculateScores(np.array(devel_logits),y_devel)
    test_end_time=time.time()
    print("Time taken for testing phase:",test_end_time-test_start_time)
    print("Test Set Scores\n")
    print("Test Accuracy:",test_accuracy,"Test Precision:",test_precision,"Test Recall:",test_recall,"\n")
    print("Development Set Scores\n")
    print("Dev Accuracy:",devel_accuracy,"Dev Precision:",devel_precision,"Dev Recall:",devel_recall)

Epoch: 0 	Test Loss: 20.91019 	 Devel Loss: 20.885563 	Train Loss: 17.651438
Epoch: 100 	Test Loss: 3.015857 	 Devel Loss: 3.0164132 	Train Loss: 2.3529463
Epoch: 200 	Test Loss: 2.6867425 	 Devel Loss: 2.6883624 	Train Loss: 2.156931
Epoch: 300 	Test Loss: 2.6681924 	 Devel Loss: 2.6705832 	Train Loss: 2.1501684
Epoch: 400 	Test Loss: 2.6669672 	 Devel Loss: 2.6691542 	Train Loss: 2.1496277
Epoch: 500 	Test Loss: 2.665239 	 Devel Loss: 2.6675081 	Train Loss: 2.1497588
Epoch: 600 	Test Loss: 2.666648 	 Devel Loss: 2.6687381 	Train Loss: 2.1499205
Epoch: 700 	Test Loss: 2.6658564 	 Devel Loss: 2.6678457 	Train Loss: 2.1498148
Epoch: 800 	Test Loss: 2.6687603 	 Devel Loss: 2.670962 	Train Loss: 2.1495836
Epoch: 900 	Test Loss: 2.668216 	 Devel Loss: 2.6707115 	Train Loss: 2.1495955
Epoch: 1000 	Test Loss: 2.667125 	 Devel Loss: 2.6693337 	Train Loss: 2.1499166
Time taken for training:  61.11264777183533
Time taken for testing phase: 0.023449420928955078
Test Set Scores

Test Accuracy: 0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
